In [1]:
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).content
soup = BeautifulSoup(result, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

cols = ['PostalCode', 'Borough', 'Neighborhood']
df=pd.DataFrame(table_contents, columns=cols)
print(df.shape)

(103, 3)


In [3]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()
df.shape

(103, 3)

In [4]:
#cant get geocoder.google to work so using csv file
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/
df_latlong = pd.read_csv('Geospatial_Coordinates.csv') # Read the csv data
df_latlong.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# Create new columns in df
df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df_cord, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df.index:
    c_idx = df_latlong['Postal Code'] == df.loc[idx, 'PostalCode']
    df.at[idx, 'Latitude'] = df_latlong.loc[c_idx, 'Latitude'].values
    df.at[idx, 'Longitude'] = df_latlong.loc[c_idx, 'Longitude'].values

In [6]:
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
